In [1]:
# imports
import numpy as np
import pandas as pd
import requests
import spotipy
import librosa
import json
import re 
import urllib.request

from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup

In [2]:
# spotify api creds
print("Enter Client ID: ")
SPOTIFY_CLIENT_ID = input()
print("Enter Client Secret: ")
SPOTIFY_CLIENT_SECRET = input()

Enter Client ID: 
Enter Client Secret: 


In [3]:
# function to get spotify token
def get_spotify_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_response = requests.post(auth_url, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })
    auth_response_data = auth_response.json()
    return auth_response_data['access_token']

In [4]:
# track details function
def get_track_details(spotify_id, token):
    sp = spotipy.Spotify(auth=token)
    track = sp.track(spotify_id)
    return track['name'], track['artists'][0]['name']

In [5]:
 
# functions to get the lyrics from spotify id
def get_lyrics_by_spotify_id(spotify_id): 

    spotify_token = get_spotify_token(SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET)
    song_title, artist = get_track_details(spotify_id, spotify_token)

    artist = artist.lower() 
    song_title = song_title.lower()

    # remove all except alphanumeric characters from artist and song_title 
    artist = re.sub('[^A-Za-z0-9]+', "", artist) 
    song_title = re.sub('[^A-Za-z0-9]+', "", song_title)

    # remove starting 'the' from artist e.g. the who -> who
    if artist.startswith("the"):     
        artist = artist[3:] 
    url = "http://azlyrics.com/lyrics/"+artist+"/"+song_title+".html"

     
    try: 
        content = urllib.request.urlopen(url).read() 
        soup = BeautifulSoup(content, 'html.parser') 
        lyrics = str(soup) 
        # lyrics lies between up_partition and down_partition 
        up_partition = '<!-- Usage of azlyrics.com content by any third-party lyrics provider is prohibited by our licensing agreement. Sorry about that. -->' 
        down_partition = '<!-- MxM banner -->' 
        lyrics = lyrics.split(up_partition)[1] 
        lyrics = lyrics.split(down_partition)[0] 
        lyrics = lyrics.replace('<br>','').replace('</br>','').replace('</div>','').replace('<br/>','').strip()
        
        return lyrics     
    except Exception as e: 
        return "Exception occurred \n" +str(e)

In [6]:
# example usage (the middle - zedd)
spotify_id = 'https://open.spotify.com/track/09IStsImFySgyp0pIQdqAc?si=31abcdbe97a641b5'
lyrics = get_lyrics_by_spotify_id(spotify_id)

print(lyrics)

Take a seat
Right over there, sat on the stairs
Stay or leave
The cabinets are bare and I'm unaware
Of just how we got into this mess
Got so aggressive
I know we meant all good intentions

So pull me closer
Why don't you pull me close
Why don't you come on over
I can't just let you go

Oh baby
Why don't you just meet me in the middle
I'm losing my mind just a little
So why don't you just meet me in the middle
In the middle
Baby
Why don't you just meet me in the middle
I'm losing my mind just a little
So why don't you just meet me in the middle
In the middle

Take a step
Back for a minute, into the kitchen
Floors are wet
And taps are still running, dishes are broken
How did we get into this mess
Got so aggressive
I know we meant all good intentions

So pull me closer
Why don't you pull me close
Why don't you come on over
I can't just let you go

Oh baby
Why don't you just meet me in the middle
I'm losing my mind just a little
So why don't you just meet me in the middle
In the middle

Lo